In [8]:
!pip install yagmail==0.15.293

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.7/385.7 kB 24.4 MB/s eta 0:00:00


In [9]:
import requests
import yagmail

# === CONFIGURATION ===

# Remplace ici par tes identifiants API
CLIENT_ID = "PAR_recherche1_ddd60b8871ceb8d358e085c054369e497bec2cf9dc795bae6af2f99ab4c13224"
CLIENT_SECRET = "3d6c9be402caed947e4d648614184a3edfef242acad45379cb57df320a6ee5d3"

# pour obtenir un clé openai https://platform.openai.com/settings/organization/api-keys
EMAIL_USER = "didier.orlandi06@gmail.com"
EMAIL_PASSWORD = "pvdrtqdjiccohnys"

DESTINATAIRES = ["didier.orlandi06@gmail.com", "didier.orlandi@wanadoo.fr"]

In [10]:
# === AUTHENTIFICATION ===
def get_token():
    url = "https://entreprise.pole-emploi.fr/connexion/oauth2/access_token?realm=/partenaire"
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = {
        "grant_type": "client_credentials",
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "scope": "api_offresdemploiv2 o2dsoffre"
    }
    r = requests.post(url, headers=headers, data=data)
    r.raise_for_status()
    return r.json()["access_token"]

# === RECHERCHE D'OFFRES ===
def chercher_offres(token):
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }
    params = {
        "motsCles": "chimie",
        "range": "0-9"
    }
    url = "https://api.pole-emploi.io/partenaire/offresdemploi/v2/offres/search"
    r = requests.get(url, headers=headers, params=params)
    r.raise_for_status()
    return r.json()

# === FORMATAGE POUR EMAIL ===
def format_offres(data):
    offres = data.get("resultats", [])
    liste = []
    for offre in offres:
        titre = offre.get("intitule", "Sans titre")
        entreprise = offre.get("entreprise", {}).get("nom", "Anonyme")
        lieu = offre.get("lieuTravail", {}).get("libelle", "N/A")
        id_offre = offre.get("id")
        lien = f"https://candidat.pole-emploi.fr/offres/recherche/detail/{id_offre}" if id_offre else "Lien indisponible"
        liste.append(f"{titre} – {entreprise} – {lieu}\n{lien}")
    return "\n\n".join(liste)

# === ENVOI D'EMAIL ===
def envoyer_email(contenu):
    print("📧 Envoi de l’email...")
    yag = yagmail.SMTP(EMAIL_USER, EMAIL_PASSWORD)
    sujet = "🧪 Offres d'emploi en chimie – hebdomadaire"
    yag.send(to=DESTINATAIRES, subject=sujet, contents=contenu)
    print("✅ Email envoyé avec succès.")

# === EXÉCUTION ===
if __name__ == "__main__":
    try:
        token = get_token()
        data = chercher_offres(token)
        resume = format_offres(data)
        print(resume)  # affichage console
        envoyer_email(resume)
    except Exception as e:
        print("❌ Erreur :", e)


Conducteur de lignes en industrie chimique F/H – SYNERGIE – 81 - Albi
https://candidat.pole-emploi.fr/offres/recherche/detail/193ZVWL

#MANE Conducteur  Industrie Chimique (H/F) – Anonyme – 06 - LE BAR SUR LOUP
https://candidat.pole-emploi.fr/offres/recherche/detail/193ZTTC

Opérateur de production Chimie (h/f) – ADECCO – 30 - Aramon
https://candidat.pole-emploi.fr/offres/recherche/detail/193ZPQX

Professeur/Professeure de physique chimie (H/F) – COFAP-CONSEIL FORMATION ASSISTANCE PROFE – 44 - Nantes
https://candidat.pole-emploi.fr/offres/recherche/detail/193ZPNB

Technicien Contrôle Physico Chimique (h/f) – LHH – 45 - Gidy
https://candidat.pole-emploi.fr/offres/recherche/detail/193ZPHY

Technicien chimiste(h/f) – ADECCO FRANCE – 60 - Compiègne
https://candidat.pole-emploi.fr/offres/recherche/detail/193ZMYG

Professeur / Professeure de physique chimie (H/F) – Collèges Lycées généraux Châteauneuf – 26 - CHATEAUNEUF DE GALAURE
https://candidat.pole-emploi.fr/offres/recherche/detail/193ZK